In [14]:
import pandas as pd
import json

# Prepare the result data

# Helper function to parse the answer
def parse_answer(ans):
    return json.loads(ans.strip("```").lstrip("json").replace("\n", "").replace(" ", "")).get("가장_적합한_HS_코드", "").rjust(6, "0")

# Load CSV
result = pd.read_csv('../results/coupang_result_v3_kor_20241208_182113.csv')

# Parse the answer
result['Result'] = result['Answer'].apply(parse_answer)

result = result.drop(columns = ['CleanDescription', 'Best3', 'Answer'])

result[:3]

,ProductName,Result
0,"품목명: 오뚜기 흰밥, 210g, 24개",100640
1,"품목명: 신라면 120g, 30개",190230
2,"품목명: 스팸 25% 라이트, 9개",150420


In [35]:
# Prepare the ground truth data

file_path = 'coupang_openai_gt_kor.csv'
ground_truth = pd.read_csv(file_path)
ground_truth = ground_truth.drop(columns = ['가장_적합한_HS_항목명', '판단_근거'])
ground_truth['가장_적합한_HS_코드'] = ground_truth['가장_적합한_HS_코드'].apply(lambda x: str(x).rjust(6, "0"))
ground_truth

,가장_적합한_HS_코드
0,190490
1,190230
2,160249
3,160414
4,220190
5,160232
6,220299
7,190220
8,080510
9,210690


In [36]:
result_10 = result.reindex([0, 1, 2, 4, 6, 8, 9, 10, 11, 13])
result_10 = result_10.reset_index(drop=True)
result_10

,ProductName,Result
0,"품목명: 오뚜기 흰밥, 210g, 24개",100640
1,"품목명: 신라면 120g, 30개",190230
2,"품목명: 스팸 25% 라이트, 9개",150420
3,"품목명: 동원 고추 참치, 85g, 8개",160419
4,"품목명: 탐사수 무라벨, 2L, 24개",220110
5,"품목명: 곰곰 더 진한 갈비탕 (냉동), 600g, 5개",020220
6,"품목명: 블루다이아몬드 아몬드 브리즈 오리지널, 190ml, 24개",080212
7,"품목명: 비비고 왕교자 (냉동), 1.05kg, 2개",160552
8,"품목명: 귤탐 당도선별 11brix 감귤, 2kg(로열과), 1개",200899
9,"품목명: 담터 호두 아몬드 율무차, 18g, 50개입, 1개",210120


In [37]:
merged_df = pd.concat([result_10, ground_truth], axis=1)
merged_df

,ProductName,Result,가장_적합한_HS_코드
0,"품목명: 오뚜기 흰밥, 210g, 24개",100640,190490
1,"품목명: 신라면 120g, 30개",190230,190230
2,"품목명: 스팸 25% 라이트, 9개",150420,160249
3,"품목명: 동원 고추 참치, 85g, 8개",160419,160414
4,"품목명: 탐사수 무라벨, 2L, 24개",220110,220190
5,"품목명: 곰곰 더 진한 갈비탕 (냉동), 600g, 5개",020220,160232
6,"품목명: 블루다이아몬드 아몬드 브리즈 오리지널, 190ml, 24개",080212,220299
7,"품목명: 비비고 왕교자 (냉동), 1.05kg, 2개",160552,190220
8,"품목명: 귤탐 당도선별 11brix 감귤, 2kg(로열과), 1개",200899,080510
9,"품목명: 담터 호두 아몬드 율무차, 18g, 50개입, 1개",210120,210690


In [40]:
for i, row in merged_df.iterrows():
    # Create a dictionary from the row
    result = merged_df.loc[i, "Result"]
    gt = merged_df.loc[i, "가장_적합한_HS_코드"]
    merged_df.loc[i, "Match_2"] = result[:2] == gt[:2]
    merged_df.loc[i, "Match_4"] = result[:4] == gt[:4]
    merged_df.loc[i, "Match_6"] = result == gt

In [41]:
merged_df

,ProductName,Result,가장_적합한_HS_코드,Match_2,Match_4,Match_6
0,"품목명: 오뚜기 흰밥, 210g, 24개",100640,190490,False,False,False
1,"품목명: 신라면 120g, 30개",190230,190230,True,True,True
2,"품목명: 스팸 25% 라이트, 9개",150420,160249,False,False,False
3,"품목명: 동원 고추 참치, 85g, 8개",160419,160414,True,True,False
4,"품목명: 탐사수 무라벨, 2L, 24개",220110,220190,True,True,False
5,"품목명: 곰곰 더 진한 갈비탕 (냉동), 600g, 5개",020220,160232,False,False,False
6,"품목명: 블루다이아몬드 아몬드 브리즈 오리지널, 190ml, 24개",080212,220299,False,False,False
7,"품목명: 비비고 왕교자 (냉동), 1.05kg, 2개",160552,190220,False,False,False
8,"품목명: 귤탐 당도선별 11brix 감귤, 2kg(로열과), 1개",200899,080510,False,False,False
9,"품목명: 담터 호두 아몬드 율무차, 18g, 50개입, 1개",210120,210690,True,False,False


In [ ]:
# 각 매치별 True 비율 계산
for match_type in ["Match_2", "Match_4", "Match_6"]:
    true_ratio = merged_df[match_type].mean()
    print(f"{match_type} True 비율: {true_ratio:.2f}")

# 전체 True 비율 계산
total_true_ratio = merged_df[["Match_2", "Match_4", "Match_6"]].values.sum() / (3 * len(merged_df))
print(f"전체 True 비율: {total_true_ratio:.2f}")

Match_2 True 비율: 0.40
Match_4 True 비율: 0.30
Match_6 True 비율: 0.10
전체 True 비율: 0.27
